In [1]:
import json
from tqdm.auto import tqdm

from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

### Load sugarcrepe

In [2]:
folder_path = '/home/drdo/vlm-compositionality/data/raw/sugarcrepe/'

# add attribute
with open (folder_path+'add_att.json') as f:
    add_attribute = json.load(f)

# add object
with open (folder_path+'add_obj.json') as f:
    add_object = json.load(f)

# replace attribute
with open (folder_path+'replace_att.json') as f:
    replace_attribute = json.load(f)

# replace object
with open (folder_path+'replace_obj.json') as f:
    replace_object = json.load(f)

# replace relation
with open (folder_path+'replace_rel.json') as f:
    replace_relation = json.load(f)

# swap attribute
with open (folder_path+'swap_att.json') as f:
    swap_attribute = json.load(f)

# swap object
with open (folder_path+'swap_obj.json') as f:
    swap_object = json.load(f)

# collate together
dataset = {
    'add_attribute': add_attribute, 'add_object': add_object, 'replace_attribute': replace_attribute,
    'replace_object': replace_object, 'replace_relation': replace_relation,
    'swap_attribute': swap_attribute, 'swap_object': swap_object,
}

### Load model

In [3]:
# model_name
model_name = "Qwen/Qwen2-VL-2B-Instruct"

# load model
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
     torch_dtype=torch.bfloat16,
     attn_implementation="flash_attention_2",
     device_map="auto",
)
model.eval()

# load processor
processor = AutoProcessor.from_pretrained(model_name)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Helper functions

In [4]:
import random
random.seed(42) # 42, 33, 56
img_folder = '/home/drdo/vlm-compositionality/data/raw/coco_val_2017/'

def get_random_sample(split_name, s_id):
    all_ids = list(dataset[split_name].keys())
    r_id = random.choice(all_ids)
    # make sure its not the same example
    while r_id == s_id:
        r_id = random.choice(all_ids)
    sample = dataset[split_name][r_id]
    return sample


def prepare_inputs(messages):
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")
    return inputs

 
def generate_text(inputs, max_new_tokens=128):
    generated_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return output_text

### Prompt functions

In [5]:
def compose_zero_shot_prompt(sample, prompt_num=1, shuffle_ordering='random'):
    
    # final prompt
    messages = []

    # prompt template

    if prompt_num == 1:
        prompt_instruct = "Choose and return the correct caption for the image from the following 2 captions. " 
        add_instruct = "Generate no other text. "
        instruct_content = "Caption 1: {} Caption 2: {}"

    elif prompt_num == 2:
        prompt_instruct = "Select the correct caption for the given image. "
        add_instruct = "Return either 'Caption 1' or 'Caption 2'. "
        instruct_content = "The captions are as follows :  Caption 1: {} Caption 2: {}"

    # sample
    caption = sample['caption']
    negative_caption = sample['negative_caption']
    captions = [caption, negative_caption]
    
    # randomly choose ordering of the captions
    if shuffle_ordering == 'random':
        c_id = random.choice([1,0])
        prompt = (prompt_instruct + add_instruct + instruct_content).format(captions[c_id], captions[1-c_id])
        label = "Caption 1" if c_id == 0 else "Caption 2"
    elif shuffle_ordering == 'first':
        prompt = (prompt_instruct + add_instruct + instruct_content).format(captions[0], captions[1])
        label = "Caption 1"
    elif shuffle_ordering == 'second':
        prompt = (prompt_instruct + add_instruct + instruct_content).format(captions[1], captions[0])
        label = "Caption 2"

    # add to content
    user_content = []
    user_content.append({"type": "image", "image": img_folder+sample['filename']}),
    user_content.append({"type": "text", "text": prompt}),

    # add to final message
    messages.append({"role": "user", "content": user_content})

    return messages, label


def compose_few_shot_prompt(split_name, s_id, sample, num_examples=1):
    
    # final prompt
    messages = []

    # prompt template

    prompt_instructs = ["Given an image your task is to return the correct caption from two given captions. ",
                        "Return the correct caption for the last image. Return either 'Caption 1' or 'Caption 2'. ", 
                        "The captions and the correct outputs for the previous images are as follows : " 
    ]
    r_samples = []
    ex_instructs = []
    for n in range(num_examples):

        # select random example
        r_sample = get_random_sample(split_name, s_id)
        r_samples.append(r_sample)
        caption = r_sample['caption']
        negative_caption = r_sample['negative_caption']
        captions = [caption, negative_caption]

        # randomly order captions
        c_id = random.choice([1,0])
        r_label = "Caption 1" if c_id == 0 else "Caption 2"

        ex_instructs.append(
            (
                "Captions for image {} : Captions 1 : {} Caption 2 : {} Correct output : {}. "
            ).format(
                n+1, captions[c_id], captions[1-c_id], r_label
            )
        )

    # last image
    caption = sample['caption']
    negative_caption = sample['negative_caption']
    captions = [caption, negative_caption]
    # randomly order captions
    c_id = random.choice([1,0])
    label = "Caption 1" if c_id == 0 else "Caption 2"
    final_instruct = ("Captions for the last image : Captions 1 : {} Caption 2 : {}").format(captions[c_id], captions[1-c_id])

    # compile prompt
    prompt = ''.join(prompt_instructs) + ''.join(ex_instructs) + final_instruct

    # add to content
    user_content = []
    for n in range(num_examples):
        user_content.append({"type": "image", "image": img_folder+r_samples[n]['filename']})
    user_content.append({"type": "image", "image": img_folder+sample['filename']})
    user_content.append({"type": "text", "text": prompt})

    # add to final message
    messages.append({"role": "user", "content": user_content})

    return messages, label

### Zero-Shot

In [ ]:
total_len = sum([len(split) for name, split in dataset.items()])
bar = tqdm(range(total_len))
incorrect_outputs = []

for split_name, split in dataset.items():
    
    accuracy = 0
    for s_id, sample in split.items():
        
        # compose prompt with chat template
        messages, label = compose_zero_shot_prompt(sample, prompt_num=1, shuffle_ordering='random')

        # prepare for inference
        inputs = prepare_inputs(messages)

        # generate text
        output_text = generate_text(inputs)

        # eval
        output_label = output_text[0]
        if output_label == label: accuracy += 1
        elif output_label not in ['Caption 1', 'Caption 2']:
            incorrect_outputs.append({'label': label, 'output': output_text})

        bar.update(1)

    print("{} : {}".format(split_name, accuracy/len(split)))

print(len(incorrect_outputs))

### Few-Shot

In [6]:
total_len = sum([len(split) for _, split in dataset.items()])
bar = tqdm(range(total_len))
incorrect_outputs = []

for split_name, split in dataset.items():
    
    accuracy = 0
    for s_id, sample in split.items():
        
        # compose prompt with chat template
        messages, label = compose_few_shot_prompt(split_name, s_id, sample, num_examples=2)

        # prepare for inference
        inputs = prepare_inputs(messages)

        # generate text
        output_text = generate_text(inputs)

        # eval
        output_label = output_text[0]
        if output_label == label: accuracy += 1
        elif output_label not in ['Caption 1', 'Caption 2']:
            incorrect_outputs.append({'label': label, 'output': output_text})

        bar.update(1)

    print("{} : {}".format(split_name, accuracy/len(split)))

print(len(incorrect_outputs))

  0%|          | 0/7511 [00:00<?, ?it/s]

add_attribute : 0.48554913294797686
add_object : 0.5237633365664404
replace_attribute : 0.48984771573604063
replace_object : 0.5338983050847458
replace_relation : 0.5056899004267426
swap_attribute : 0.506006006006006


#### Testing prompts

In [30]:
img_folder = '/home/drdo/vlm-compositionality/data/raw/coco_val_2017/'
example = swap_object['2']

In [33]:
img_file = img_folder+example['filename']
caption = example['caption']
negative_caption = example['negative_caption']

In [34]:
example

{'filename': '000000287347.jpg',
 'caption': 'A woman prepares a pizza while a man watches.',
 'negative_caption': 'A man prepares a pizza while a woman watches.'}

In [25]:
swap_object['3']

{'filename': '000000177934.jpg',
 'caption': 'Girls wash a motorcycle while men look on.',
 'negative_caption': 'Men wash a motorcycle while girls look on.'}

In [ ]:
#chat = [

  #{"role": "user", "content": "Hello, how are you?"},

  #{"role": "assistant", "content": "I'm doing great. How can I help you today?"},

  #{"role": "user", "content": "I'd like to show off how chat templating works!"},

#]

In [7]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": img_file,},
            {"type": "text", "text": "Choose and return the more appropriate caption for the image from the following 2 captions : 1) "+negative_caption+" 2) "+caption+" Generate no other text."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['1) A man prepares a pizza while a woman watches.']


In [46]:
len(messages)

1

In [45]:
messages[0].keys()

dict_keys(['role', 'content'])

In [ ]:
# observation

# shuffle ordering
# few(even 1) shot helps -> need to make sure example do not contain answer
# how to ensure consistent output?

In [36]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": img_folder+'000000051309.jpg',},
            {"type": "text", "text": "Choose and return the more appropriate caption for the image from the following 2 captions. Generate no other text. Caption 1: Three large horses eating hay while a small horse stands behind. Caption 2: A small horse eating hay while three large horses stand behind ."},
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "Caption 1"}
        ]
    },
     {
        "role": "user",
        "content": [
            {"type": "image", "image": img_folder+'000000177934.jpg',},
            {"type": "text", "text": "Choose and return the more appropriate caption for the image from the following 2 captions. Generate no other text. Caption 1: Men wash a motorcycle while girls look on. Caption 2: Girls wash a motorcycle while men look on."},
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "Caption 1"}
        ]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": img_file,},
            {"type": "text", "text": "Choose and return the more appropriate caption for the image from the following 2 captions. Generate no other text. Caption 1: A man prepares a pizza while a woman watches. Caption 2: A woman prepares a pizza while a man watches."},
        ]
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['Caption 1']


In [57]:
messages

[{'role': 'user',
  'content': [{'type': 'image',
    'image': '/home/drdo/vlm-compositionality/data/raw/coco_val_2017/000000287347.jpg'},
   {'type': 'text',
    'text': 'Choose and return the more appropriate caption for the image from the following 2 captions. Generate no other text. Caption 1: A man prepares a pizza while a woman watches. Caption 2: A woman prepares a pizza while a man watches.'}]}]